In [1]:
import torch
import numpy as np
import torchvision.models as models
import torch.nn as nn
model = models.vgg11(pretrained=True)
model_control =models.vgg11(pretrained=True)


In [2]:
from PIL import Image
import torchvision.transforms as transforms

In [3]:
# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
def process_image(np_im):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    np_im=np_im.reshape(240,240,3)
    transformations = transforms.Compose([transforms.ToTensor()])
    torch_image = transformations(np_im).float()    
    return torch_image

In [5]:
def predict(np_im):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''

    
    if use_cuda:
        model.cuda()
    
    model.eval()    
    
    with torch.no_grad():
       
        image = process_image(np_im)
        if use_cuda:
            image = image.type(torch.FloatTensor).cuda()   
        #I do not understand why we shold do this why not 3 dimensions are not sufficent.
        #print(image.shape)
        image = image.unsqueeze(0)
        #print(image.shape)
        
        output = model.forward(image)
        output =output.cpu().detach().numpy()
        #ps = torch.exp(logps)
        #_,cls_idx=torch.max(output, 1)
        #probs_tensor, classes_tensor = ps.topk(1,dim=1)

    #return probs_tensor, classes_tensor    
    #return cls_idx.item()
    
    return output

 ## Values are same because it is a pretrained model!

In [6]:
model.classifier[3].weight[5]

tensor([-0.0095, -0.0086, -0.0099,  ...,  0.0050,  0.0056,  0.0020],
       grad_fn=<SelectBackward>)

In [7]:
model_control.classifier[3].weight[5]

tensor([-0.0095, -0.0086, -0.0099,  ...,  0.0050,  0.0056,  0.0020],
       grad_fn=<SelectBackward>)

In [15]:
for param in model.features.parameters():
    param.requires_grad = False
    
 
#number thrusters on the Lilly   
thruster=4
first_layer= nn.Linear(25088, 2048)
model.classifier[0] = first_layer
# new layers automatically have requires_grad = True
second_layer= nn.Linear(2048, 1024)
model.classifier[3] = second_layer
last_layer = nn.Linear(1024, thruster)
model.classifier[6] = last_layer

In [16]:
for param in model_control.features.parameters():
    param.requires_grad = False
    

#number thrusters on the Lilly   
thruster=4
first_layer= nn.Linear(25088, 2048)
model_control.classifier[0] = first_layer
# new layers automatically have requires_grad = True
second_layer= nn.Linear(2048, 1024)
model_control.classifier[3] = second_layer
last_layer = nn.Linear(1024, thruster)
model_control.classifier[6] = last_layer

In [17]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print("cudaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
print(use_cuda)
# move model to GPU if CUDA is available
if use_cuda:
    model = model.cuda()

cudaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
True


## Now different

In [19]:
model.classifier[3].weight[0]

tensor([ 0.0131,  0.0163, -0.0194,  ...,  0.0122,  0.0039,  0.0187],
       device='cuda:0', grad_fn=<SelectBackward>)

In [20]:
model_control.classifier[3].weight[0]


tensor([-0.0138,  0.0185,  0.0055,  ...,  0.0070, -0.0084, -0.0160],
       grad_fn=<SelectBackward>)

In [21]:
model.classifier[3].bias[5]

tensor(-0.0162, device='cuda:0', grad_fn=<SelectBackward>)

In [22]:
model_control.classifier[3].bias[5]

tensor(0.0067, grad_fn=<SelectBackward>)

In [23]:
model_control.features[6].weight[4].size()

torch.Size([128, 3, 3])

# Easy to transfer the weights this way from one agent to another

https://discuss.pytorch.org/t/set-model-weights-to-preset-tensor-with-torch/35051

In [24]:
for param_cur, param_best in zip(model.parameters(), model_control.parameters()):
    print(param_cur)
    param_cur.data = param_best.data

Parameter containing:
tensor([[[[ 0.2882,  0.0358, -0.3850],
          [ 0.1795,  0.3668, -0.5012],
          [-0.0974,  0.3648, -0.2296]],

         [[ 0.4015, -0.0461, -0.6842],
          [ 0.4442,  0.4478, -0.7949],
          [ 0.1129,  0.4917, -0.3705]],

         [[ 0.2162, -0.0290, -0.3949],
          [ 0.1490,  0.2967, -0.4294],
          [-0.0095,  0.3479, -0.1558]]],


        [[[-0.3528, -0.2549,  0.6734],
          [-0.6027, -0.3453,  0.8054],
          [-0.4197, -0.1347,  0.6622]],

         [[-0.5740, -0.3998,  0.7708],
          [-0.8758, -0.3668,  1.1098],
          [-0.5186, -0.0801,  0.9228]],

         [[-0.0280, -0.2954,  0.2159],
          [-0.1868, -0.2904,  0.3808],
          [-0.0926, -0.0766,  0.3727]]],


        [[[ 0.0650, -0.2592, -0.2389],
          [ 0.3209,  0.2152, -0.2047],
          [-0.0443,  0.2227, -0.0452]],

         [[ 0.0290, -0.4797, -0.5490],
          [ 0.5363,  0.2973, -0.3728],
          [ 0.1776,  0.4145, -0.0436]],

         [[ 0.0033, -0

Parameter containing:
tensor([ 6.9663e-02, -7.4578e-02,  6.8191e-02, -1.1545e-01,  1.1413e-01,
         1.6240e-01,  1.6822e-02,  1.6960e-02, -7.9377e-02, -7.3284e-02,
         5.6664e-02, -1.6892e-01,  2.1476e-01,  6.6745e-02,  5.6914e-03,
         3.0928e-01, -5.5799e-02, -7.8213e-02, -1.4124e-02,  1.2880e-01,
        -1.2512e-01,  1.0197e-01,  5.6811e-02,  1.4456e-01,  1.3809e-01,
        -3.8199e-02, -1.0181e-01,  9.0443e-02, -7.5123e-02,  2.5778e-02,
         1.7336e-01,  4.7478e-02,  9.5542e-02,  1.2848e-01, -6.8323e-02,
         3.1092e-02,  8.7274e-02,  9.9804e-02, -1.7249e-01, -1.3187e-02,
         2.0692e-01,  1.1857e-01, -9.7738e-02,  7.3444e-02,  1.3827e-01,
         1.8019e-01, -2.1991e-01,  2.1799e-01,  1.1570e-01,  7.8062e-02,
        -7.5075e-02,  1.3981e-01,  1.7306e-02,  3.4516e-01, -8.5906e-02,
        -1.5916e-01,  6.6126e-02,  1.4557e-02,  1.1967e-01, -1.2218e-02,
         2.0329e-01, -1.5549e-02,  2.3962e-01,  2.1813e-02,  4.2624e-02,
         1.4101e-01,  1.3442e

Parameter containing:
tensor([-1.4608e-02,  1.8701e-01, -6.8343e-02,  2.2371e-02, -3.5337e-02,
        -6.0920e-02, -8.3485e-03,  1.0452e-01, -5.3740e-02,  7.1650e-03,
         9.0111e-02,  1.3977e-01, -7.2416e-03,  1.1278e-01, -1.4603e-02,
         1.0699e-02, -1.0749e-01,  8.0407e-02, -4.8478e-02, -1.0573e-01,
         9.0333e-02, -1.1993e-01,  2.3968e-02,  7.1396e-02,  1.9301e-01,
         2.0070e-02,  2.0030e-02,  1.6289e-02, -9.2568e-02, -4.9245e-03,
         1.3531e-02,  5.8468e-02, -6.7849e-03,  3.0915e-02,  2.8822e-02,
        -2.1878e-01,  9.0613e-02, -1.3258e-01,  5.3929e-02, -5.7984e-02,
        -4.6772e-03, -2.4499e-01, -1.8270e-01,  3.1799e-02, -3.2034e-02,
        -5.9157e-02,  1.6989e-01,  5.1798e-02,  1.1004e-01, -6.7835e-02,
         9.8269e-03, -1.4662e-02, -2.3305e-02,  3.6911e-02,  2.7078e-01,
        -6.5170e-02,  7.4908e-02, -1.5090e-01,  7.1047e-02, -8.4295e-02,
        -1.3351e-01,  6.7799e-02,  5.2526e-02, -3.4434e-03,  2.2369e-02,
         5.9156e-02,  8.1834e

Parameter containing:
tensor([[[[ 1.2251e-03, -1.2390e-02,  1.7509e-03],
          [-4.1748e-03, -5.3147e-03, -1.0618e-02],
          [-1.0449e-02, -9.7556e-03, -3.1804e-02]],

         [[-1.9989e-02, -9.5576e-03, -1.1438e-02],
          [-2.3691e-02, -1.7388e-02, -1.5332e-02],
          [-1.1302e-02, -1.4268e-02, -1.9718e-02]],

         [[-5.8479e-03, -8.0334e-03, -1.4213e-02],
          [ 3.3119e-02,  2.9173e-02,  6.3408e-03],
          [ 6.3005e-02,  6.1142e-02,  2.8725e-02]],

         ...,

         [[ 1.3314e-02,  1.3193e-02,  2.5801e-02],
          [-6.9312e-03,  1.1561e-02,  4.5584e-03],
          [ 4.8843e-03,  5.3159e-05,  4.6373e-03]],

         [[-3.6992e-02, -3.4953e-02, -4.1560e-02],
          [-3.3384e-02, -2.9443e-02, -3.7354e-02],
          [-6.5742e-03, -1.7729e-02, -9.9308e-03]],

         [[-1.8839e-02, -2.9188e-02, -2.2033e-02],
          [-1.1977e-02, -2.2783e-02, -1.4805e-02],
          [-9.1168e-03, -1.6219e-02, -1.7485e-02]]],


        [[[-2.3495e-02, -2.6272

Parameter containing:
tensor([-1.3256e-02, -1.2854e-01, -2.1685e-02,  4.0101e-02,  1.4428e-01,
         3.5453e-02,  1.6866e-01,  3.6682e-02, -5.3320e-02,  1.2475e-01,
        -9.6327e-03, -2.1028e-02,  2.1276e-01,  2.3974e-01,  5.6999e-02,
         1.8073e-01,  8.1418e-02,  3.7219e-01,  1.5219e-01, -1.1172e-02,
        -1.6110e-03,  1.3036e-01,  9.1855e-04,  9.6962e-02, -1.1623e-02,
         1.7353e-01,  1.0665e-01,  3.0280e-02, -6.1670e-02,  1.7710e-01,
        -1.1142e-02,  7.0902e-02,  4.1434e-02,  1.3642e-02,  1.4336e-01,
         6.7812e-02,  6.3456e-02,  2.1378e-01,  9.8502e-02,  2.5382e-01,
         1.6177e-01, -6.1809e-02,  1.6945e-01, -4.1712e-01,  2.4145e-01,
        -7.2108e-03,  9.8799e-02,  2.9377e-01,  1.9277e-01,  2.1125e-01,
         1.6118e-02,  5.3348e-02, -1.3928e-02,  2.4986e-01,  5.6474e-02,
         1.8636e-01,  6.8500e-02,  2.4383e-02,  1.0469e-01,  3.3546e-02,
         1.9247e-01, -1.2226e-01, -4.1193e-02,  5.6616e-02, -3.9151e-02,
         3.7914e-02,  1.8338e

## Now same again

In [25]:
model_control.classifier[6].weight[0]

tensor([ 0.0240, -0.0147,  0.0100,  ...,  0.0220,  0.0251, -0.0297],
       grad_fn=<SelectBackward>)

In [26]:
model.classifier[6].weight[0]

tensor([ 0.0240, -0.0147,  0.0100,  ...,  0.0220,  0.0251, -0.0297],
       grad_fn=<SelectBackward>)

## Sizes

In [27]:
model.classifier[3].weight.size()

torch.Size([1024, 2048])

In [84]:
model.classifier[3].weight[0].size()

torch.Size([2048])

## Modules in the model

In [128]:
for param in model.classifier.modules():
    print(param.)
    

<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>


In [28]:
for param in model.classifier:
    print(type(param))

<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>


In [55]:
model.classifier[6].weight[3].size()

torch.Size([1024])

In [42]:
model.classifier[0].in_features

25088

In [46]:
model.classifier[0].bias.data

tensor([ 0.0014, -0.0041,  0.0040,  ..., -0.0013, -0.0023,  0.0047])

In [114]:
type(model.classifier[0])

torch.nn.modules.linear.Linear

In [57]:
torch.rand(1)

tensor([0.6314])

In [16]:
model.hidden.bias

Parameter containing:
tensor([-0.0121, -0.0149,  0.0135, -0.0033, -0.0219, -0.0047,  0.0140,  0.0191,
         0.0054, -0.0284, -0.0082, -0.0229, -0.0201, -0.0233,  0.0101, -0.0329,
         0.0132, -0.0218,  0.0117, -0.0169, -0.0047, -0.0035,  0.0121,  0.0196,
        -0.0271, -0.0186,  0.0111,  0.0248,  0.0161,  0.0150, -0.0066, -0.0313,
        -0.0086, -0.0078,  0.0353,  0.0084, -0.0024, -0.0317,  0.0217, -0.0001,
        -0.0064, -0.0008, -0.0109, -0.0329,  0.0075,  0.0104,  0.0260, -0.0310,
        -0.0105, -0.0170, -0.0343, -0.0069,  0.0212,  0.0124,  0.0123,  0.0229,
        -0.0165, -0.0290,  0.0150, -0.0038,  0.0196, -0.0097, -0.0327, -0.0345,
         0.0079, -0.0115, -0.0192, -0.0265,  0.0108,  0.0160,  0.0134,  0.0262,
         0.0062,  0.0189,  0.0059, -0.0295, -0.0074, -0.0251,  0.0091,  0.0182,
         0.0141,  0.0293, -0.0345, -0.0129,  0.0255, -0.0068,  0.0256,  0.0199,
        -0.0049, -0.0208, -0.0321, -0.0018,  0.0243, -0.0052, -0.0183,  0.0206,
         0.0323, -